In [0]:
%sql
-- Create SQL Procedure for sp_ADS_SMA_CPP_MAIN_PREPARE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_PROCESS_EVENTS_MAIN(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for sp_ADS_SMA_PROCESS_EVENTS_MAIN'
AS
BEGIN
    -- Declare local variables
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_SMA_PROC',
            step_name => 'ADS_SMA_PROCESS_EVENTS_MAIN',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'PROCESS_EVENTS',
            log_id => my_log_id,
            p_load_date => p_load_date
        );
    
    /* Fill Stage Table */
    TRUNCATE TABLE gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE;

    insert into gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE
    SELECT
        eventid,
        type,
        cidla,
        get_json_object(jsondata, '$.timestamp') AS timestamp,
        get_json_object(jsondata, '$.user') AS usera,
        get_json_object(jsondata, '$.takeType') AS takeType,
        get_json_object(jsondata, '$.serviceId') AS serviceId,
        get_json_object(jsondata, '$.clientTask.id') AS clientTaskid,
        get_json_object(jsondata, '$.clientActivity.id') AS clientActivityid,
        get_json_object(jsondata, '$.humanTask.cluid') AS humanTaskcluid,
        get_json_object(jsondata, '$.humanTask.id') AS humanTaskid,
        get_json_object(jsondata, '$.theCase.cluid') AS theCasecluid,
        get_json_object(jsondata, '$.georgeTask.cluid') AS georgeTaskcluid,
        get_json_object(jsondata, '$.georgeTask.id') AS georgeTaskid,
        get_json_object(jsondata, '$.humanTask.label') AS label
    FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full j
    WHERE j.SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE) 
    ;
    
    /* Fill XC Table */
    TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_SMA_PROCESS_EVENTS_MAIN;    
    INSERT INTO gap_catalog.ads_etl_owner.XC_SMA_PROCESS_EVENTS_MAIN 
        (
            EVE_SOURCE_ID                                 -- Process event source id
        , EVE_SOURCE_SYSTEM_ID                          -- Process event source system id
        , EVE_SOURCE_SYS_ORIGIN                         -- Origin (Source table)
        , PT_KEY                                        -- Party warehouse key - (potential) client.
        , PTORI_KEY                                     -- Surrogate key of origin - (potential) client.
        , PT_UNIFIED_KEY                                -- Unified Party warehouse key of the (potential) client.
        , PT_EMPLOYEE_KEY                               -- Party warehouse key - employee, i.e. typically the processor of the event (see also the table description)
        , PTORI_EMPLOYEE_KEY                            -- Surrogate key of origin - employee
        , ID_FINANCIAL_PRODUCT                          -- Unique identifier of Financial Product (surrogate)
        , CASEPH_KEY                                    -- CASE_PHASES primary generated identifier (surrogate key).
        , NTP_KEY                                       -- NOTIFICATION_TYPES primary generated identifier (surrogate key).
        , CHAN_KEY                                      -- CHANNELS primary generated identifier (surrogate key).
        , EVETP_KEY                                     -- EVENT_TYPES primary generated identifier - event type.
        , EST_KEY                                       -- EVENT_STATUS primary generated identifier (surrogate key).
        , CASE_KEY                                      -- CASE primary generated identifier (surrogate key).
        , EVE_DATE                                      -- Event date - business date, actual date of the event
        , EVE_DATETIME                                  -- Event date with time - business date, actual date of event
        , EVE_RECORDED_DATE                             -- Date, when the event was recorded in the source system
        , PROCE_TARGET                                  -- Target on which the event was executed (e.g. mail, telephone etc)
        , STR_KEY                                       -- STATUS_REASON primary generated identifier (surrogate key).
        , CGP_ID                                        -- Bank-wide unique product instance identifier (from RTODS) - FK to ASWBL.BD_PRODUCTS
        , ACC_KEY                                       -- ACCOUNTS key
        , ACCTP_KEY                                     -- ACCOUNT_TYPES key
        , CARD_KEY                                      -- CARDS key 
        , SRV_KEY                                       -- Services key
        , CASE_MASTER_KEY                               -- (MASTER) CASE primary generated identifier (surrogate key).
        , DOC_KEY                                       -- Link into DWH_OWNER.DOCUMENTS
        , DOCL_KEY                                      -- Surrogate key into DOCUMENT_LOCATIONS
        ,EVE_STATUS_DESC                                -- Status description
        )
        SELECT       
            EVE_SOURCE_ID                           
        , EVE_SOURCE_SYSTEM_ID                    
        , EVE_SOURCE_SYS_ORIGIN                   
        , PT_KEY                                  
        , PTORI_KEY                               
        , PT_UNIFIED_KEY                          
        , PT_EMPLOYEE_KEY                         
        , PTORI_EMPLOYEE_KEY                      
        , ID_FINANCIAL_PRODUCT                    
        , CASEPH_KEY                              
        , NTP_KEY                                 
        , CHAN_KEY                                
        , EVETP_KEY                               
        , EST_KEY                                 
        , CASE_KEY                                
        , EVE_DATE                                
        , EVE_DATETIME                            
        , EVE_RECORDED_DATE                       
        , PROCE_TARGET                            
        , STR_KEY                                 
        , CGP_ID                                  
        , ACC_KEY                                 
        , ACCTP_KEY                               
        , CARD_KEY                                
        , SRV_KEY                                 
        , CASE_MASTER_KEY                         
        , DOC_KEY                                 
        , DOCL_KEY  
        ,EVE_STATUS_DESC                              
    FROM (
                with ca0 as (
                select 
                s.eventid,
                to_date(from_unixtime(TIMESTAMP/1000)) as dt_timestamp,
                to_date(from_unixtime(TIMESTAMP/1000)) as t_timestamp,
                s.type type,
                s.cidla,
                cp.caseph_key,
                CP.case_key,
                s.USER,
                HUMANTASKCLUID AS CLUID,
                TAKETYPE as taketype,
                null as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                left join gap_catalog.ads_src_20250901.case_phases cp on s.HUMANTASKID = cp.caseph_source_id
                and  cp.caseph_source_system_id = 'SMA'
                and  cp.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
                and cp.cpcat_key=2 and cp.CASEPH_VALID_TO=date'3000-01-01'
                where
                s.type like ('HT_%') or
                s.type in ('REQUISITION_CREATED','REQUISITION_COMPLETED')
                union  all
                select 
                s.eventid,
                to_date(from_unixtime(TIMESTAMP/1000)) dt_timestamp,
                to_date(from_unixtime(TIMESTAMP/1000)) t_timestamp,
                s.type type,
                s.cidla,
                a.caseph_key as caseph_key,
                nvl(a.case_key, -1) as case_key,
                s.USER,
                coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
                TAKETYPE as taketype,
                null as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                left join gap_catalog.ads_src_20250901.case_phases a 
                on a.caseph_source_id= s.SERVICEID
                and  a.caseph_source_system_id = 'SMA'
                and  a.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
                and a.cpcat_key=6 and a.CASEPH_VALID_TO=date'3000-01-01'
                --EVENT filter
                where s.type like ('ACTIVITY_%')
                union  all
                select
                s.eventid,
                to_date(from_unixtime(TIMESTAMP/1000)) dt_timestamp,
                to_date(from_unixtime(TIMESTAMP/1000)) t_timestamp,
                s.type type,
                s.cidla,
                CT.caseph_key as caseph_key,
                nvl(CT.case_key, -1) as case_key,
                s.USER,
                coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
                TAKETYPE as taketype,
                null as eve_status_desc              
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                left join gap_catalog.ads_src_20250901.case_phases ct on CLIENTTASKID = ct.caseph_source_id
                and  ct.caseph_source_system_id = 'SMA'
                and  ct.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
                and ct.cpcat_key=7 and ct.CASEPH_VALID_TO=date'3000-01-01'
                --EVENT filter
                where s.type like ('CLIENT_TASK%')
                union  all
                select 
                s.eventid,
                to_date(from_unixtime(TIMESTAMP/1000)) dt_timestamp,
                to_date(from_unixtime(TIMESTAMP/1000)) t_timestamp,
                s.type type,
                s.cidla,
                Ca.caseph_key as caseph_key,
                nvl(CA.case_key, -1) as case_key,
                s.USER,
                coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
                TAKETYPE as taketype,
                null as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                left join gap_catalog.ads_src_20250901.case_phases ca on s.CLIENTACTIVITYID = ca.caseph_source_id
                and  ca.caseph_source_system_id = 'SMA'
                and  ca.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
                and ca.cpcat_key=8 and ca.CASEPH_VALID_TO=date'3000-01-01'
                --EVENT filter
                where s.type like ('CLIENT_ACTIVITY%')
                union  all
                select
                s.eventid,
                to_date(from_unixtime(s.TIMESTAMP/1000)) dt_timestamp,
                to_date(from_unixtime(s.TIMESTAMP/1000))  t_timestamp,
                s.type type,
                s.cidla,
                GT.caseph_key as caseph_key,
                nvl(GT.case_key, -1) as case_key,
                s.USER,
                coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
                s.TAKETYPE as taketype,
                null as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                left join gap_catalog.ads_src_20250901.case_phases gt on s.GEORGETASKID = gt.caseph_source_id
                and  gt.caseph_source_system_id = 'SMA'
                and  gt.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
                and gt.cpcat_key=314 and gt.CASEPH_VALID_TO=date'3000-01-01' 
                --EVENT filter
                where s.type like ('GEORGE%')
                union all
                select 
                    s.eventid,
                    to_date(from_unixtime(s.TIMESTAMP/1000)) dt_timestamp,
                    to_date(from_unixtime(s.TIMESTAMP/1000))  t_timestamp,
                    s.type type,
                    s.cidla,
                    -1 as caseph_key,
                    NULL case_key,
                    s.USER,
                    coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
                    s.TAKETYPE as taketype,
                s.label as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                --EVENT filter
                where  
                S.TYPE LIKE ('RUNNER%')
            union all
                select 
                    s.eventid,
                    to_date(from_unixtime(s.TIMESTAMP/1000)) dt_timestamp,
                    to_date(from_unixtime(s.TIMESTAMP/1000))  t_timestamp,
                    s.type type,
                    s.cidla,
                    -1 as caseph_key,
                    NULL case_key,
                    s.USER,
                    coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
                    s.TAKETYPE as taketype,
                null as eve_status_desc
                from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
                --EVENT filter
                where s.type like ('CASE%') 
                ) 
                select
                ca0.eventid as eve_source_id,
                'SMA' as eve_source_system_id,
                'SMA_MONITOR_EVENTS' as eve_source_sys_origin,
                nvl(P1.PT_KEY, -1) as pt_key,
                nvl(P1.PTORI_KEY, -1) as ptori_key,
                nvl(p1.PT_UNIFIED_KEY, -1) as pt_unified_key,
                nvl(p.PT_KEY,-1) as pt_employee_key,
                nvl(p.PTORI_KEY,-1) as ptori_employee_key,
                'XNA' as id_financial_product,
                ca0.caseph_key,
                -1 as ntp_key,
                -1 as chan_key,
                nvl(et.evetp_key, -1) as evetp_key,
                -1 as est_key,
                coalesce(c.case_key, ca0.case_key, -1) as case_key,
                DATE(ca0.dt_timestamp) as eve_date,
                ca0.t_timestamp as eve_datetime,
                ca0.t_timestamp as eve_recorded_date,
                'XNA' as proce_target,
                nvl(sr.str_key, -1) as str_key,
                nvl(a.CGP_ID, 'XNA') as cgp_id, nvl(a.ACC_KEY, -1) as acc_key, nvl(a.ACCTP_KEY, -1) as acctp_key,
                nvl(a.CARD_KEY, -1) as card_key, nvl(a.SRV_KEY, -1) as srv_key, nvl(c.CASE_MASTER_KEY, -1) as case_master_key, -1 as doc_key,
                -1 as docl_key ,
                eve_status_desc
                from ca0
                left join gap_catalog.ads_src_20250901.cases c on ca0.cidla = c.case_source_id  and c.case_source_system_id = 'SMA' and c.case_source_sys_origin = 'SMA_MONITOR_EVENTS' and c.case_valid_to=date'3000-01-01'
                left join gap_catalog.dwh_owner.parties p on upper(ca0.USER)=p.pt_source_id and p.ptori_key = 32 and p.pt_source_system_id='LD0'
                left join gap_catalog.dwh_owner.parties p1 on p1.PT_UNIQUE_PARTY_ID = ca0.cluid and p1.ptori_key = 4 and p1.pt_source_system_id='CR0'
                left join gap_catalog.ads_src_20250901.event_types et on et.evetp_source_id = ca0.type and et.evetp_source_system_id = 'SMA' and  et.evetp_source_sys_origin = 'SMA_SYSTEM_EVENTS' and et.evetp_valid_to=date'3000-01-01' 
                left join gap_catalog.ads_src_20250901.status_reason sr on ca0.taketype= sr.STR_SOURCE_ID and  sr.STR_SOURCE_SYSTEM_ID='SMA' and  sr.STR_SOURCE_SYS_ORIGIN='SMA_SYSTEM_EVENTS.TAKE_TYPE' and  sr.STR_VALID_TO =date'3000-01-01'
                left join (SELECT 
                    PROD_INST_CASES.CASE_KEY AS CASE_KEY ,
                    PROD_INST_CASES.ACC_KEY AS ACC_KEY ,
                    PROD_INST_CASES.ACCTP_KEY AS ACCTP_KEY ,
                    PROD_INST_CASES.CARD_KEY AS CARD_KEY ,
                    PROD_INST_CASES.SRV_KEY AS SRV_KEY ,
                    PROD_INST_CASES.CGP_ID AS CGP_ID ,
                    rank() over (partition by PROD_INST_CASES.CASE_KEY order by PROD_INST_CASES.ACC_KEY desc, PROD_INST_CASES.CARD_KEY desc, case when PROD_INST_CASES.CGP_ID <> 'XNA' then 0 else 10 end, PROD_INST_CASES.PRICASE_VALID_FROM) AS rn   
                FROM 
                    gap_catalog.ads_src_20250901.PROD_INST_CASES PROD_INST_CASES  INNER JOIN  gap_catalog.ads_src_20250901.PROD_INST_CASE_TYPES PROD_INST_CASE_TYPES  
                    ON  PROD_INST_CASES.PRICASETP_KEY = PROD_INST_CASE_TYPES.PRICASETP_KEY
                WHERE
                    (CAST(p_load_date AS DATE) between PROD_INST_CASES.PRICASE_VALID_from and PROD_INST_CASES.PRICASE_VALID_TO
                ) AND   (PROD_INST_CASE_TYPES.PRICASETP_VALID_TO = date '3000-01-01'
                and PROD_INST_CASE_TYPES.PRICASETP_SOURCE_SYSTEM_ID = 'ADS'
                and PROD_INST_CASE_TYPES.PRICASETP_SOURCE_ID = 'SMA_CASE_CGP')) a 
                on c.case_key=a.case_key and a.rn=1
    );  
 /* Target Merge */
        MERGE INTO gap_catalog.ads_owner.PROCESS_EVENTS trg
        USING
        (
        SELECT EVE_SOURCE_ID, 
        EVE_SOURCE_SYSTEM_ID, 
        EVE_SOURCE_SYS_ORIGIN, 
        PT_KEY, 
        PTORI_KEY, 
        PT_UNIFIED_KEY, 
        PT_EMPLOYEE_KEY, 
        PTORI_EMPLOYEE_KEY, 
        ID_FINANCIAL_PRODUCT, 
        CASEPH_KEY, 
        NTP_KEY, 
        CHAN_KEY, 
        EVETP_KEY, 
        EST_KEY, 
        CASE_KEY, 
        CGP_ID, 
        EVE_DATE, 
        EVE_DATETIME, 
        EVE_RECORDED_DATE, 
        PROCE_TARGET, 
        ACC_KEY, 
        ACCTP_KEY, 
        CARD_KEY, 
        SRV_KEY, 
        CASE_MASTER_KEY, 
        DOC_KEY, 
        DOCL_KEY, 
        STR_KEY, 
        EVE_STATUS_DESC
        FROM gap_catalog.ads_etl_owner.XC_SMA_PROCESS_EVENTS_MAIN xc
        WHERE (EVE_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
            AND EVE_DATE IN (CAST(p_load_date AS DATE), 
                             CAST(p_load_date AS DATE) - 1, 
                             CAST(p_load_date AS DATE) + 1)
            AND EVE_SOURCE_SYSTEM_ID = 'SMA'
            AND EVETP_KEY IN (SELECT EVETP_KEY 
                                FROM gap_catalog.ads_src_20250901.EVENT_TYPES
                                WHERE EVENT_TYPES.EVETP_VALID_TO = to_date('30000101','yyyyMMdd')
                                AND EVENT_TYPES.EVETP_SOURCE_SYSTEM_ID = 'SMA' 
                                AND EVENT_TYPES.EVETP_SOURCE_SYS_ORIGIN = 'SMA_SYSTEM_EVENTS'))
        ) src
        ON (trg.EVE_SOURCE_ID = src.EVE_SOURCE_ID
            AND trg.EVE_SOURCE_SYSTEM_ID = src.EVE_SOURCE_SYSTEM_ID
            AND trg.EVE_SOURCE_SYS_ORIGIN = src.EVE_SOURCE_SYS_ORIGIN
            AND trg.EVETP_KEY = src.EVETP_KEY
            AND trg.EVE_DATE = src.EVE_DATE)
        WHEN MATCHED AND (
            NOT (src.PT_KEY <=> trg.PT_KEY) OR
            NOT (src.PTORI_KEY <=> trg.PTORI_KEY) OR
            NOT (src.PT_UNIFIED_KEY <=> trg.PT_UNIFIED_KEY) OR
            NOT (src.PT_EMPLOYEE_KEY <=> trg.PT_EMPLOYEE_KEY) OR
            NOT (src.PTORI_EMPLOYEE_KEY <=> trg.PTORI_EMPLOYEE_KEY) OR
            NOT (src.ID_FINANCIAL_PRODUCT <=> trg.ID_FINANCIAL_PRODUCT) OR
            NOT (src.CASEPH_KEY <=> trg.CASEPH_KEY) OR
            NOT (src.NTP_KEY <=> trg.NTP_KEY) OR
            NOT (src.CHAN_KEY <=> trg.CHAN_KEY) OR
            NOT (src.EST_KEY <=> trg.EST_KEY) OR
            NOT (src.CASE_KEY <=> trg.CASE_KEY) OR
            NOT (src.CGP_ID <=> trg.CGP_ID) OR
            NOT (src.EVE_DATETIME <=> trg.EVE_DATETIME) OR
            NOT (src.EVE_RECORDED_DATE <=> trg.EVE_RECORDED_DATE) OR
            NOT (src.PROCE_TARGET <=> trg.PROCE_TARGET) OR
            NOT (src.ACC_KEY <=> trg.ACC_KEY) OR
            NOT (src.ACCTP_KEY <=> trg.ACCTP_KEY) OR
            NOT (src.CARD_KEY <=> trg.CARD_KEY) OR
            NOT (src.SRV_KEY <=> trg.SRV_KEY) OR
            NOT (src.CASE_MASTER_KEY <=> trg.CASE_MASTER_KEY) OR
            NOT (src.DOC_KEY <=> trg.DOC_KEY) OR
            NOT (src.DOCL_KEY <=> trg.DOCL_KEY) OR
            NOT (src.STR_KEY <=> trg.STR_KEY) OR
            NOT (src.EVE_STATUS_DESC <=> trg.EVE_STATUS_DESC)
        ) THEN UPDATE SET 
            trg.PT_KEY = src.PT_KEY, 
            trg.PTORI_KEY = src.PTORI_KEY, 
            trg.PT_UNIFIED_KEY = src.PT_UNIFIED_KEY, 
            trg.PT_EMPLOYEE_KEY = src.PT_EMPLOYEE_KEY, 
            trg.PTORI_EMPLOYEE_KEY = src.PTORI_EMPLOYEE_KEY, 
            trg.ID_FINANCIAL_PRODUCT = src.ID_FINANCIAL_PRODUCT, 
            trg.CASEPH_KEY = src.CASEPH_KEY, 
            trg.NTP_KEY = src.NTP_KEY, 
            trg.CHAN_KEY = src.CHAN_KEY, 
            trg.EST_KEY = src.EST_KEY, 
            trg.CASE_KEY = src.CASE_KEY, 
            trg.CGP_ID = src.CGP_ID, 
            trg.EVE_DATETIME = src.EVE_DATETIME, 
            trg.EVE_RECORDED_DATE = src.EVE_RECORDED_DATE, 
            trg.PROCE_TARGET = src.PROCE_TARGET, 
            trg.PROCE_UPDATED_DATETIME = CURRENT_TIMESTAMP(), 
            trg.PROCE_UPDATE_PROCESS_KEY = p_process_key, 
            trg.ACC_KEY = src.ACC_KEY, 
            trg.ACCTP_KEY = src.ACCTP_KEY, 
            trg.CARD_KEY = src.CARD_KEY, 
            trg.SRV_KEY = src.SRV_KEY, 
            trg.CASE_MASTER_KEY = src.CASE_MASTER_KEY, 
            trg.DOC_KEY = src.DOC_KEY, 
            trg.DOCL_KEY = src.DOCL_KEY, 
            trg.STR_KEY = src.STR_KEY, 
            trg.EVE_STATUS_DESC = src.EVE_STATUS_DESC
        WHEN NOT MATCHED THEN INSERT 
        (EVE_KEY, 
        EVE_SOURCE_ID, 
        EVE_SOURCE_SYSTEM_ID, 
        EVE_SOURCE_SYS_ORIGIN, 
        PT_KEY, 
        PTORI_KEY, 
        PT_UNIFIED_KEY, 
        PT_EMPLOYEE_KEY, 
        PTORI_EMPLOYEE_KEY, 
        ID_FINANCIAL_PRODUCT, 
        CASEPH_KEY, 
        NTP_KEY, 
        CHAN_KEY, 
        EVETP_KEY, 
        EST_KEY, 
        CASE_KEY, 
        CGP_ID, 
        EVE_DATE, 
        EVE_DATETIME, 
        EVE_RECORDED_DATE, 
        PROCE_TARGET, 
        PROCE_INSERTED_DATETIME, 
        PROCE_INSERT_PROCESS_KEY, 
        PROCE_UPDATED_DATETIME, 
        PROCE_UPDATE_PROCESS_KEY, 
        ACC_KEY, 
        ACCTP_KEY, 
        CARD_KEY, 
        SRV_KEY, 
        CASE_MASTER_KEY, 
        DOC_KEY, 
        DOCL_KEY, 
        STR_KEY, 
        EVE_STATUS_DESC
        ) 
        VALUES 
        (xxhash64(uuid()),
        src.EVE_SOURCE_ID, 
        src.EVE_SOURCE_SYSTEM_ID, 
        src.EVE_SOURCE_SYS_ORIGIN, 
        src.PT_KEY, 
        src.PTORI_KEY, 
        src.PT_UNIFIED_KEY, 
        src.PT_EMPLOYEE_KEY, 
        src.PTORI_EMPLOYEE_KEY, 
        src.ID_FINANCIAL_PRODUCT, 
        src.CASEPH_KEY, 
        src.NTP_KEY, 
        src.CHAN_KEY, 
        src.EVETP_KEY, 
        src.EST_KEY, 
        src.CASE_KEY, 
        src.CGP_ID, 
        src.EVE_DATE, 
        src.EVE_DATETIME, 
        src.EVE_RECORDED_DATE, 
        src.PROCE_TARGET, 
        CURRENT_TIMESTAMP(), 
        p_process_key, 
        CURRENT_TIMESTAMP(), 
        p_process_key, 
        src.ACC_KEY, 
        src.ACCTP_KEY, 
        src.CARD_KEY, 
        src.SRV_KEY, 
        src.CASE_MASTER_KEY, 
        src.DOC_KEY, 
        src.DOCL_KEY, 
        src.STR_KEY, 
        src.EVE_STATUS_DESC)
        ;

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

END;
END;